(In order to load the stylesheet of this notebook, execute the last code cell in this notebook)

# Stack Overflow

## Introduction 

In this assignment, we will look at some posts on Stack Overflow during the year of 2015 and measure the similarity of users by looking at the types of questions they answer. We will also analyze the creation dates of questions.

## Step 0. Preparation

Before we start working on the notebook, let's make sure that everything is setup properly. You should have downloaded and installed
* [Anaconda](https://store.continuum.io/cshop/anaconda/)
* [Git](http://git-scm.com/downloads)

If you are working from the undergraduate lab (on a linux machine) these are both installed, but you need to follow the instructions [from here](https://github.com/datascience16/lectures/blob/master/Lecture2/Getting-Started.ipynb).



## Step 1. Getting the data

Let's make a sample request to retrieve the questions posted on Stack Exchange on the first day of 2015. Documentation of the Stack Exchange API can be found [here](https://api.stackexchange.com/docs).

In [14]:
import requests

start_time = 1420070400 # 01-01-2015 at 00:00:00
end_time   = 1420156800 # 01-02-2015 at 00:00:00

response = requests.get("https://api.stackexchange.com/2.2/questions?pagesize=100" +
                        "&fromdate=" + str(start_time) + "&todate=" + str(end_time) +
                        "&order=asc&sort=creation&site=stackoverflow")
print response

<Response [200]>


All dates in the Stack Exchange API are in [unix epoch time](https://en.wikipedia.org/wiki/Unix_time). The format for the request string is specified [here](https://api.stackexchange.com/docs/questions).

We can try to print the response that Stack Exchange returns.

In [3]:
print response.text

It is not possible to read the raw response. Instead, we need to decode the raw response as JSON and use the `json` library to print it.

In [2]:
import json

print json.dumps(response.json(), indent=2)

Now we can easily see that the response consists of a list of question items. For each of these items, we get information about its attributes such as its `creation_date`, `answer_count`, `owner`, `title`, etc.

Notice that has_more is true. To get more items, we can [request the next page](https://api.stackexchange.com/docs/paging).

-----------------

## Step 2. Parsing the responses

In this section, we practice some of the basic Python tools that we learned in class and the powerful string handling methods that Python offers. Our goal is to be able to pick the interesting parts of the response and transform them in a format that will be useful to us.

First let's isolate the creation_date in the response. Fill in the rest of the ```print_creation_dates_json()``` function that reads the response and prints the creation dates. Notice that a JSON object is basically a dictionary. **(5 pts)**

In [4]:
import time, datetime
def print_creation_dates_json(response):
    data = json.loads(response.text)
    items = data['items']
    for element in items:
        #print element['creation_date']
        print datetime.datetime.utcfromtimestamp(int(element['creation_date']))

Write the code that calls the ```print_creation_dates_json()``` function to print out all the creation dates of questions posted on the first day in 2015. Please be aware of Stack Exchange's [rate limit](https://api.stackexchange.com/docs/throttle). **(5 pts)**

In [5]:
#1420088400
import requests
import json
import time

def pullAllPages(response):
    data = json.loads(response.text)
    print_creation_dates_json(response)
    page = 2
    while(data['has_more'] == True):
        start_time = 1420070400 # 01-01-2015 at 00:00:00
        end_time   = 1420156800 # 01-02-2015 at 00:00:00
        time.sleep(6)
        response = requests.get("https://api.stackexchange.com/2.2/questions?pagesize=100&page=" + str(page) +
                        "&fromdate=" + str(start_time) + "&todate=" + str(end_time) +
                        "&order=asc&sort=creation&site=stackoverflow")
        page = page + 1
        data = json.loads(response.text)
        print_creation_dates_json(response)
    
response = requests.get("https://api.stackexchange.com/2.2/questions?pagesize=100&page=1" +
                        "&fromdate=1420070400&todate=1420156800&order=asc&sort=creation&site=stackoverflow")

pullAllPages(response)

2015-01-01 00:00:58
2015-01-01 00:01:43
2015-01-01 00:02:32
2015-01-01 00:02:57
2015-01-01 00:03:31
2015-01-01 00:04:01
2015-01-01 00:05:03
2015-01-01 00:05:27
2015-01-01 00:05:34
2015-01-01 00:06:17
2015-01-01 00:06:41
2015-01-01 00:07:28
2015-01-01 00:07:39
2015-01-01 00:07:46
2015-01-01 00:09:28
2015-01-01 00:10:05
2015-01-01 00:10:29
2015-01-01 00:11:43
2015-01-01 00:12:02
2015-01-01 00:12:55
2015-01-01 00:13:04
2015-01-01 00:13:32
2015-01-01 00:13:50
2015-01-01 00:15:40
2015-01-01 00:17:11
2015-01-01 00:18:50
2015-01-01 00:22:16
2015-01-01 00:23:14
2015-01-01 00:23:50
2015-01-01 00:24:28
2015-01-01 00:25:07
2015-01-01 00:25:29
2015-01-01 00:25:39
2015-01-01 00:26:42
2015-01-01 00:27:01
2015-01-01 00:27:54
2015-01-01 00:28:49
2015-01-01 00:30:43
2015-01-01 00:32:22
2015-01-01 00:32:34
2015-01-01 00:33:17
2015-01-01 00:33:50
2015-01-01 00:34:15
2015-01-01 00:34:41
2015-01-01 00:36:50
2015-01-01 00:37:18
2015-01-01 00:37:47
2015-01-01 00:38:05
2015-01-01 00:39:37
2015-01-01 00:39:39


KeyboardInterrupt: 

Due to time constraints, we have downloaded the [data dump](http://cs-people.bu.edu/kzhao/teaching/stackoverflow-posts-2015.tar.gz) for Stack Overflow's posts in 2015. Note that this file is 10GB. If you don't have space on your computer, you can download it into `/scratch` on one of the machines in the undergrad lab or you can download it onto a USB. You may also want to work with a subset of this data at first, but your solution should be efficient enough to work with the whole dataset. For example, if you call `read()` on this file, you will get a `MemoryError`.

Write a function to parse out the questions posted in 2015. These are posts with `PostTypeId=1`. Make a `pandas DataFrame` with 3 columns: `Id`, `CreationDate`, `OwnerUserId`, and the first tag in `Tags`. Save the `DataFrame` to a file named `question_dataframe.csv` using `to_csv()`. **(10 pts)**

In [28]:
import pandas as pd
import xml.etree.ElementTree as etree

def fetchData():
    #df = pd.DataFrame(columns=('Id', 'CreationDate', 'OwnerUserId', 'Tag'))
    my_dict = {}
    count = 0
    for event, elem in etree.iterparse('stackoverflow-posts-2015.xml'):
        #if(count > 2000):
            #break
        try:
            if(elem.attrib['PostTypeId'] == '1'):
                tag = elem.attrib['Tags']
                tag = tag.split('>')[0][1:]
                Id = elem.attrib['Id']
                date = elem.attrib['CreationDate']
                try:
                    owner = elem.attrib['OwnerUserId']
                except:
                    owner = elem.attrib['OwnerDisplayName']
            
                list1 = [Id, date, owner,tag]
                my_dict[count] = list1
                list1 = []
                count = count + 1
            elem.clear()
        except Exception,e: print str(e)
    #print my_dict
    df = pd.DataFrame(my_dict) 
    res = df.transpose()
    res.columns = ['id', 'date', 'ownerId', 'tag']
    res.to_csv('question_dataframe.csv')
    
        
        
    

In [60]:
import pandas as pd
fetchData()
#question_df = pd.read_csv('question_dataframe_sample.csv')
question_df = pd.read_csv('question_dataframe.csv')
print question_df

'PostTypeId'
         Unnamed: 0        id                     date     ownerId  \
0                 0  27727385  2015-01-01T00:00:58.253     3210431   
1                 1  27727388  2015-01-01T00:01:43.673      868779   
2                 2  27727391  2015-01-01T00:02:32.123     4372672   
3                 3  27727393  2015-01-01T00:02:57.983     2482149   
4                 4  27727394  2015-01-01T00:03:31.337     4263870   
5                 5  27727396  2015-01-01T00:04:01.407     4409381   
6                 6  27727406  2015-01-01T00:05:03.773      875317   
7                 7  27727407  2015-01-01T00:05:27.167      821742   
8                 8  27727408  2015-01-01T00:05:34.733     2595033   
9                 9  27727409  2015-01-01T00:06:17.720     1815395   
10               10  27727410  2015-01-01T00:06:41.067      541091   
11               11  27727414  2015-01-01T00:07:28.747     1210038   
12               12  27727418  2015-01-01T00:07:39.243     3674356   
13     

-----------------

## Step 3. Putting it all together

We are now ready to tackle our original problem. Write a function to measure the similarity of the top 1000 users with the most answer posts. Compare the users based on the types of questions they answer. We will categorize the questions by looking at the first tag in each question. You may choose to implement any one of the similarity/distance measures we discussed in class. Document your findings. **(30pts)**

Note that answers are posts with `PostTypeId=2`. The ID of the question in answer posts is the `ParentId`.

You may find the [sklearn.feature_extraction module](http://scikit-learn.org/stable/modules/feature_extraction.html) helpful.

In [1]:
import pandas as pd
import xml.etree.ElementTree as etree
def fetchAnswerPostsData():
    count = 0;
    tmp_dict = {}
    for event, elem in etree.iterparse('stackoverflow-posts-2015.xml'):
        #if(count > 1000):
            #break
        try:
            if(elem.attrib['PostTypeId'] == '2'):
                try:
                    owner = elem.attrib['OwnerUserId']
                except:
                    owner = elem.attrib['OwnerDisplayName']
                Id = elem.attrib['Id']
                parent = elem.attrib['ParentId']
                buff = [Id, owner, parent]
                tmp_dict[count] = buff
                count = count + 1
            elem.clear()
        except Exception,e: print str(e)
    df = pd.DataFrame(tmp_dict)  
    ans = df.transpose()
    ans.columns = ['id','ownerId','parentId']
    ans.to_csv('answer_dataframe.csv', encoding='utf-8')
    print 'end'
    return
 
def top_Users():
    answer_df = pd.read_csv('answer_dataframe.csv')
    answer_df = answer_df.groupby(['ownerId']).count()
    #make a new copy
    res = answer_df['parentId'].copy()
    #sort by descending order
    res.sort(ascending = False)
    #fetch the top 1000 users on the list
    res = res.head(1000)
    print res
    return res
    
def users_questions_tags(list):
    
#fetchAnswerPostsData()
top_Users()

ownerId
1144035    7573
548225     4303
3732271    3392
3297613    3101
1491895    2954
100297     2865
3832970    2797
2141635    2540
114251     2414
771848     2400
6309       2359
115145     2353
1221571    2310
157247     2310
434551     2189
335858     2094
207421     2009
22656      1959
992484     1943
19068      1911
341994     1878
795990     1852
2422776    1841
816620     1718
2435473    1694
462627     1685
4039065    1663
1501794    1588
2025923    1557
2877241    1557
           ... 
4302471     260
192373      260
3695849     260
2571212     259
4363119     259
1346234     259
4628565     259
960558      259
1860929     259
4687135     259
696808      259
76051       258
283366      258
294248      258
5299236     258
1464112     258
363751      258
4851590     258
3970411     257
354577      257
2060725     257
2405040     257
3478010     257
3282633     257
1683264     257
33258       257
779513      256
1672429     256
33518       256
4053652     256
Name: parentId, 

/Users/yili/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:35: FutureWarning: sort is deprecated, use sort_values(inplace=True) for for INPLACE sorting


ownerId
1144035    7573
548225     4303
3732271    3392
3297613    3101
1491895    2954
100297     2865
3832970    2797
2141635    2540
114251     2414
771848     2400
6309       2359
115145     2353
1221571    2310
157247     2310
434551     2189
335858     2094
207421     2009
22656      1959
992484     1943
19068      1911
341994     1878
795990     1852
2422776    1841
816620     1718
2435473    1694
462627     1685
4039065    1663
1501794    1588
2025923    1557
2877241    1557
           ... 
4302471     260
192373      260
3695849     260
2571212     259
4363119     259
1346234     259
4628565     259
960558      259
1860929     259
4687135     259
696808      259
76051       258
283366      258
294248      258
5299236     258
1464112     258
363751      258
4851590     258
3970411     257
354577      257
2060725     257
2405040     257
3478010     257
3282633     257
1683264     257
33258       257
779513      256
1672429     256
33518       256
4053652     256
Name: parentId, 

Plot the top 100 most similar users. See [Lecture 3](https://github.com/datascience16/lectures/blob/master/Lecture3/Distance-Functions.ipynb) for examples. **(10 pts)**

In [55]:
import pandas as pd
#fetchAnswerPostsData()


Next, let's create some time series from the data. Look at the top 100 users with the most question posts. For each user, your time series will be the `CreationDate` of the questions posted by that user. You may want to make multiple time series for each user based on the first tag of the questions. Compare the time series using one of the methods discussed in class. Document your findings. **(30 pts)**

You may find the [pandas.DataFrame.resample module](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.resample.html) helpful.

Plot the 2 most similar and the 2 most different time series. **(10 pts)**

In [ ]:
# Code for setting the style of the notebook
from IPython.core.display import HTML
def css_styling():
    styles = open("../theme/custom.css", "r").read()
    return HTML(styles)
css_styling()